In [3]:
import pandas as pd
import oracledb

# Minhas credenciais
credencial_banco = {
    'dsn': 'oracle.fiap.com.br:1521/orcl',
    'usuario': 'rm553472',
    'senha': '100593'
}

# Função para conectar ao banco de dados
def conectar(usuario, senha, dsn):
    try:
        conexao = oracledb.connect(user=usuario, password=senha, dsn=dsn, mode=oracledb.DEFAULT_AUTH)
        print("Conexão com o banco de dados Oracle estabelecida com sucesso.")
        return conexao
    except oracledb.DatabaseError as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
        return None

In [4]:
# Conexão com o banco de dados para obter feedback
conexao_feedback = conectar(usuario='rm553472', senha='100593', dsn='oracle.fiap.com.br:1521/orcl')

# Consulta para obter feedback
query_feedback = """
SELECT 
    f.id_feedback, 
    f.fk_id_cliente, 
    f.fk_id_especialista, 
    f.fk_id_clinica, 
    f.nota, 
    f.comentario,
    c.nome_completo AS nome_cliente,
    e.nome AS nome_dentista,
    cl.nome AS nome_clinica
FROM Feedback f
JOIN Cliente c ON f.fk_id_cliente = c.id_cliente
JOIN Especialista e ON f.fk_id_especialista = e.id_especialista
JOIN Clinica cl ON f.fk_id_clinica = cl.id_clinica
ORDER BY f.id_feedback
"""

# Carregar feedback em um DataFrame
df_feedback = pd.read_sql(query_feedback, conexao_feedback)

# Fechar a conexão com o banco de dados
conexao_feedback.close()


Conexão com o banco de dados Oracle estabelecida com sucesso.


/var/folders/38/6x4f55x100b7srd3m42bprm80000gn/T/ipykernel_19135/1904180862.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_feedback = pd.read_sql(query_feedback, conexao_feedback)


In [5]:
df_feedback

,ID_FEEDBACK,FK_ID_CLIENTE,FK_ID_ESPECIALISTA,FK_ID_CLINICA,NOTA,COMENTARIO,NOME_CLIENTE,NOME_DENTISTA,NOME_CLINICA
0,1,1,1,1,5,Excelente atendimento!,Carlos Silva,Dr. João Silva,Clínica Saúde e Sorriso
1,2,2,2,2,4,"Bom serviço, mas o tempo de espera foi longo.",Maria Oliveira,Dra. Maria Oliveira,Clínica OdontoPlus
2,3,3,3,3,3,"Atendimento razoável, poderia ser melhor.",João Pereira,Dr. Carlos Santos,Clínica Bem Estar
3,4,4,4,4,2,Não fiquei satisfeito com o atendimento.,Ana Costa,Dra. Ana Costa,Clínica DentalCare
4,5,5,5,5,1,"Péssimo serviço, não recomendo.",Lucas Santos,Dr. Pedro Almeida,Clínica Sorriso Feliz
...,...,...,...,...,...,...,...,...,...
198,199,3,1,3,2,"Consulta decepcionante, serviço ruim.",João Pereira,Dr. João Silva,Clínica Bem Estar
199,200,3,2,3,2,"O atendimento foi fraco, fiquei insatisfeito.",João Pereira,Dra. Maria Oliveira,Clínica Bem Estar
200,201,4,3,3,2,"Não gostei da consulta, serviço ruim.",Ana Costa,Dr. Carlos Santos,Clínica Bem Estar
201,202,4,1,1,2,"Foi um atendimento ruim, faltou atenção.",Ana Costa,Dr. João Silva,Clínica Saúde e Sorriso


In [6]:
# Agrupar feedbacks por clínica e calcular a média das notas
df_feedback_agrupado = df_feedback.groupby('FK_ID_CLINICA').agg(
    media_nota=('NOTA', 'mean'),  # Média das notas
    total_feedbacks=('ID_FEEDBACK', 'count')  # Contagem total de feedbacks
).reset_index()

# Renomear a coluna para o ID da clínica
df_feedback_agrupado.rename(columns={'FK_ID_CLINICA': 'ID_CLINICA'}, inplace=True)


In [7]:
df_feedback_agrupado

,ID_CLINICA,media_nota,total_feedbacks
0,1,2.987654,81
1,2,2.966102,59
2,3,2.964286,56
3,4,2.000000,1
4,5,1.000000,1
5,6,5.000000,1
6,7,4.000000,1
7,8,3.000000,1
8,9,5.000000,1
9,10,2.000000,1


In [8]:
# Filtrar clínicas com média de feedbacks positivos
df_clinicas_com_feedback = df_feedback_agrupado[df_feedback_agrupado['media_nota'] >= 4]
df_clinicas_com_feedback

,ID_CLINICA,media_nota,total_feedbacks
5,6,5.0,1
6,7,4.0,1
8,9,5.0,1


In [9]:
# Selecionar a melhor clínica (com a maior média de notas ou com mais feedbacks)
melhor_clinica = df_clinicas_com_feedback.sort_values(by='media_nota', ascending=False).iloc[0] if not df_clinicas_com_feedback.empty else None
melhor_clinica

if melhor_clinica is not None:
    print("Melhor Clínica Sugerida:")
    print(melhor_clinica)
else:
    print("Nenhuma clínica encontrada que atenda aos critérios.")


Melhor Clínica Sugerida:
ID_CLINICA         6.0
media_nota         5.0
total_feedbacks    1.0
Name: 5, dtype: float64


## Se não tiver clinicas o sufuciente, ai a nota de corte será 3 para cima.

Nunca as de nota 1 e 2.

In [10]:
# Filtrar clínicas com média de feedbacks positivos
df_clinicas_com_feedback_tres = df_feedback_agrupado[df_feedback_agrupado['media_nota'] >= 3]
df_clinicas_com_feedback_tres

,ID_CLINICA,media_nota,total_feedbacks
5,6,5.0,1
6,7,4.0,1
7,8,3.0,1
8,9,5.0,1


In [11]:
# Selecionar a melhor clínica (com a maior média de notas ou com mais feedbacks)
melhor_clinica = df_clinicas_com_feedback.sort_values(by='media_nota', ascending=False).iloc[0] if not df_clinicas_com_feedback.empty else None
melhor_clinica

if melhor_clinica is not None:
    print("Melhor Clínica Sugerida:")
    print(melhor_clinica)
else:
    print("Nenhuma clínica encontrada que atenda aos critérios.")


Melhor Clínica Sugerida:
ID_CLINICA         6.0
media_nota         5.0
total_feedbacks    1.0
Name: 5, dtype: float64
